In [1]:
#!pip install beautifulsoup4

In [2]:
import pandas as pd
import re

import requests
from bs4 import BeautifulSoup

### Regex

In [21]:
texto = """
O telefone da farmácia é 3000-3232, mas eles só atendem até as 18h em dias úteis.
Se precisar entrar em contato fora desse horário, você pode enviar uma mensagem para o celular de emergência no número (11) 98765-4321.
Importante ressaltar que hoje, 27/02/2024, é o último dia para aproveitar a promoção especial de medicamentos que começou em 01/02/2024.
Não esqueça de apresentar seu CPF na hora da compra, o número 1.78923.456-00 é um exemplo válido.
Para mais informações, acesse nosso site ou entre em contato diretamente
(12)98765-4321
27-02-2024.
"""

In [4]:
pattern = r'\d{2}/\d{2}/\d{4}|\d{2}-\d{2}-\d{4}'
re.findall(pattern, texto)

['27/02/2024', '01/02/2024', '27-02-2024']

In [5]:
pattern = r'\(?\d{2}\)\s?\d+-\d{4,5}|\d+-\d{4,5}'

re.findall(pattern, texto)

['3000-3232', '(11) 98765-4321', '(12)98765-4321', '02-2024']

### BeautifulSoup

In [6]:
url = 'https://www.google.com'

In [7]:
response = requests.get(url)
response.status_code

200

In [8]:
soup = BeautifulSoup(response.content)
links = soup.find_all('a')

for i in links:
    print(i.get('href'))


https://www.google.com/imghp?hl=pt-BR&tab=wi
https://maps.google.com.br/maps?hl=pt-BR&tab=wl
https://play.google.com/?hl=pt-BR&tab=w8
https://www.youtube.com/?tab=w1
https://news.google.com/?tab=wn
https://mail.google.com/mail/?tab=wm
https://drive.google.com/?tab=wo
https://www.google.com.br/intl/pt-BR/about/products?tab=wh
http://www.google.com.br/history/optout?hl=pt-BR
/preferences?hl=pt-BR
https://accounts.google.com/ServiceLogin?hl=pt-BR&passive=true&continue=https://www.google.com/&ec=GAZAAQ
/advanced_search?hl=pt-BR&authuser=0
/intl/pt-BR/ads/
/services/
/intl/pt-BR/about.html
https://www.google.com/setprefdomain?prefdom=BR&prev=https://www.google.com.br/&sig=K_WSEr4w_VwJYi-Ag6S5W7flHEvUY%3D
/intl/pt-BR/policies/privacy/
/intl/pt-BR/policies/terms/


##### Webscraping Yahoo

In [9]:
'''
Alguns sites restringem o acesso a seus recursos quando as requisições são feitas por bots ou scripts automatizados. 
Configurar o header User-Agent em sua requisição pode ajudar a contornar essas restrições, simulando uma requisição feita por um navegador comum.
'''
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

r = requests.get('https://br.financas.yahoo.com/noticias/acoes-mais-negociadas/', headers=headers)
print(r)

<Response [200]>


In [10]:
r = requests.get('https://br.financas.yahoo.com/noticias/acoes-mais-negociadas/')
print(r)

<Response [404]>


In [12]:
def extrair_informacoes_acoes(url):
    """
    Extrai informações das ações mais negociadas do Yahoo Finanças.
    
    Args:
    url (str): URL da página do Yahoo Finanças com as ações mais negociadas.
    
    Returns:
    list: Uma lista de dicionários contendo as informações extraídas.
    """
    # Define headers para simular uma requisição de um navegador web
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'
    }

    # Faz a requisição para o site
    response = requests.get(url, headers=headers)
    
    # Verifica se a requisição foi bem sucedida
    if response.status_code == 200:
        # Parseia o conteúdo HTML da página
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontra todos os elementos 'tbody' da página, que contém os dados
        all_data = soup.find_all('tbody')
        stock_data = []
        
        for stocks in all_data:
            # Encontra todos os elementos 'tr', que correspondem às linhas da tabela
            rows = stocks.find_all('tr')
            for row in rows:
                # Encontra todos os elementos 'td', que correspondem às células
                cells = row.find_all('td')
                if len(cells) > 7:  # Verifica se temos o número esperado de células
                    # Extrai os dados de interesse
                    data = {
                        'stock': cells[0].get_text(),
                        'company': cells[1].get_text(),
                        'change': cells[3].get_text(),
                        'perc_change': cells[4].get_text(),
                        'vol_mean_3m': cells[6].get_text(),
                        'market_cap': cells[7].get_text()
                    }
                    stock_data.append(data)
        return stock_data
    else:
        print(f"Erro {response.status_code}: Não foi possível acessar a página.")
        return None

In [13]:
acoes_url = 'https://br.financas.yahoo.com/noticias/acoes-mais-negociadas/'
acoes_mais_negociadas = extrair_informacoes_acoes(acoes_url)

In [14]:
acoes_mais_negociadas

[{'stock': 'MGLU3.SA',
  'company': 'Magazine Luiza S.A.',
  'change': '+0,0900',
  'perc_change': '+4,27%',
  'vol_mean_3m': '165,526M',
  'market_cap': '15,74B'},
 {'stock': 'HAPV3.SA',
  'company': 'Hapvida Participações e Investimentos S.A.',
  'change': '+0,1700',
  'perc_change': '+4,75%',
  'vol_mean_3m': '71,527M',
  'market_cap': '28,107B'},
 {'stock': 'BBDC4.SA',
  'company': 'Banco Bradesco S.A.',
  'change': '+0,22',
  'perc_change': '+1,59%',
  'vol_mean_3m': '47,791M',
  'market_cap': '141,676B'},
 {'stock': 'B3SA3.SA',
  'company': 'B3 S.A. - Brasil, Bolsa, Balcão',
  'change': '+0,27',
  'perc_change': '+2,16%',
  'vol_mean_3m': '35,137M',
  'market_cap': '71,334B'},
 {'stock': 'COGN3.SA',
  'company': 'Cogna Educação S.A.',
  'change': '+0,1700',
  'perc_change': '+7,14%',
  'vol_mean_3m': '26,984M',
  'market_cap': '4,773B'},
 {'stock': 'BRFS3.SA',
  'company': 'BRF S.A.',
  'change': '+1,14',
  'perc_change': '+8,14%',
  'vol_mean_3m': '8,131M',
  'market_cap': '25,3

In [15]:
df_acoes = pd.DataFrame(acoes_mais_negociadas)
df_acoes

,stock,company,change,perc_change,vol_mean_3m,market_cap
0,MGLU3.SA,Magazine Luiza S.A.,"+0,0900","+4,27%","165,526M","15,74B"
1,HAPV3.SA,Hapvida Participações e Investimentos S.A.,"+0,1700","+4,75%","71,527M","28,107B"
2,BBDC4.SA,Banco Bradesco S.A.,"+0,22","+1,59%","47,791M","141,676B"
3,B3SA3.SA,"B3 S.A. - Brasil, Bolsa, Balcão","+0,27","+2,16%","35,137M","71,334B"
4,COGN3.SA,Cogna Educação S.A.,"+0,1700","+7,14%","26,984M","4,773B"
5,BRFS3.SA,BRF S.A.,"+1,14","+8,14%","8,131M","25,317B"
6,FNAM11.SA,Fundo de Investimento Amazônia - FINAM,"-0,0100","-3,12%",N/A,N/A
7,ABEV3.SA,Ambev S.A.,"+0,27","+2,07%","20,702M","209,625B"
8,VALE3.SA,Vale S.A.,"+1,73","+2,63%","24,611M","289,099B"
9,PETR4.SA,Petróleo Brasileiro S.A. - Petrobras,"-0,06","-0,14%","35,359M","561,322B"


### Exercícios em Sala

1. Escreva um programa que verifica se uma determinada string é um endereço de e-mail válido (caracteres_alfanuméricos@domínio.com).

In [16]:
def verifica_email(email: str)-> bool:
    #pattern = r'^\w+@\w+\.\w+$|^\w+@\w+\.\w+\.\w+$'
    pattern = r'^[\w.-]+@[\w-]+\.[\w.-]+$'
    resp = bool(re.match(pattern, email))
    return resp

2. Escreva um programa que encontre todas as ocorrências de datas no formato "dd/mm/aaaa" em uma determinada string.

In [17]:
texto = "As datas de hoje e amanhã são 25/05/2023 e 26/05/2023, respectivamente."

def encontra_data(string):
    #pattern = r'\d{2}/\d{2}/\d{4}'
    pattern = r'\d{2}/\d{2}/\d{4}'
    resp = re.findall(pattern, string)
    return resp

texto = "As datas de hoje e amanhã são 25/05/2023 e 26/05/2023, respectivamente."
encontra_data(texto)

['25/05/2023', '26/05/2023']

3. Escreva um programa que conte o número de imagens em uma página HTML.

In [18]:
def contar_imagens(html_content):
     # Realiza a requisição GET para a URL fornecida
    response = requests.get(url)
    
    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Analisa o HTML da página usando BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontra todos os elementos de imagem na página
        imagens = soup.find_all('img')
        
        # Retorna a quantidade de imagens encontradas
        return len(imagens)
    else:
        # Retorna None se a requisição não foi bem-sucedida
        return None

In [19]:
contar_imagens(url)

2